3 doors, car prize behind one of them<br>
you pick a door but can't open yet<br>
Monty (the host of the game) then opens one of the doors, always revealing no car (he knows, where it is)<br>
You now get to switch doors if you want. Should you?<br>
Intuitive reasoning: No, because 3 doors --> 1/3 chance of picking a car.
Bayes: Yes, always. 

In [32]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

# define the network structure
# (a, b) from node a to node b
model = BayesianModel([('Me', 'Host'), ('Prize', 'Host')])

CPD = conditional probability distribution<br>
TabularCPD(node, cardinality, values)<br>
cardinality = number of unique values for a given variable<br>
values = list of lists, insert a whole row<br>
from top to bottom

In [33]:
# defining the CPDs
Me_cpd = TabularCPD('Me', 3, [[0.33, 0.33, 0.33]])
Prize_cpd = TabularCPD('Prize', 3, [[0.33, 0.33, 0.33]])
Host_cpd = TabularCPD('Host', 3, [[0, 0, 0, 0, 0.5, 1, 0, 1, 0.5],
                                  [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5],
                                  [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0]], evidence=['Prize','Me'], evidence_card = [3,3])

# associate CPDs with the network
model.add_cpds(Me_cpd, Prize_cpd, Host_cpd)
print(Host_cpd)

╒════════╤═════════╤═════════╤═════════╤═════════╤═════════╤═════════╤═════════╤═════════╤═════════╕
│ Prize  │ Prize_0 │ Prize_0 │ Prize_0 │ Prize_1 │ Prize_1 │ Prize_1 │ Prize_2 │ Prize_2 │ Prize_2 │
├────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ Me     │ Me_0    │ Me_1    │ Me_2    │ Me_0    │ Me_1    │ Me_2    │ Me_0    │ Me_1    │ Me_2    │
├────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ Host_0 │ 0.0     │ 0.0     │ 0.0     │ 0.0     │ 0.5     │ 1.0     │ 0.0     │ 1.0     │ 0.5     │
├────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ Host_1 │ 0.5     │ 0.0     │ 1.0     │ 0.0     │ 0.0     │ 0.0     │ 1.0     │ 0.0     │ 0.5     │
├────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ Host_2 │ 0.5     │ 1.0     │ 0.0     │ 1.0     │ 0.5     │ 0.0     │ 0.0     │ 0.0     │ 

In [34]:
# infering posterior probability with algo called VariableElimination
from pgmpy.inference import VariableElimination
infer = VariableElimination(model)

Find P(Prize_i | evidence)

In [35]:
# wlog Host = 3, Me = 2
posterior = infer.query(variables=['Prize'], evidence={'Host': 2, 'Me': 1})
print(posterior['Prize'])

╒═════════╤══════════════╕
│ Prize   │   phi(Prize) │
╞═════════╪══════════════╡
│ Prize_0 │       0.6667 │
├─────────┼──────────────┤
│ Prize_1 │       0.3333 │
├─────────┼──────────────┤
│ Prize_2 │       0.0000 │
╘═════════╧══════════════╛


Without loss of generality, we picked door 2 and Host opened door 3<br>
and the P(Prize_1 | evidence) = 2/3, so we should always switch.

In [37]:
from pgmpy.inference import BeliefPropagation
infer_bp = BeliefPropagation(model)

In [38]:
infer_bp.query(variables=['Prize'], evidence={'Host':2, 'Me':1})

KeyError: 0